# Árbol de decisión

Según el siguiente árbol de decisión, analice y responda:

![Obama-Clinton decision tree](https://github.com/justmarkham/DAT8/raw/226791169b1cc6df8e8845c12e34e748d5ffaa85/notebooks/images/obama_clinton_tree.jpg)

## 1.1 ¿Qué son las observaciones? ¿Cuántas observaciones hay?


Son la cantidad de votos a favor y en contra de cada candidato a la presidencia de USA. En total hay 2241 observaciones.

## 1.2 ¿Cuál es la variable de respuesta?


La variable respuesta es el ganador en ciertos estados teniendo en cuenta la cantidad de condados a favor.

## 1.3 ¿Cuáles son las características?¿Cuál es la característica más predictiva?


La caracteristica más predictiva es si en el estado mas del 20% de los habitantes es de descendencia afroamericana. Con respecto a las otras caracteristicas, tenemos:
- Si el porcentaje de graduados en las secundarias es mayor al 78%
- Si el porcentaje de graduados en las secundarias es mayor al 87%
- La ubicación georafica del condado (Noreste/Sur u Oeste/medioeste)
- Durante los 2000, ¿muchos hogares del condado se encontraban en situación en pobreza? (menos de 30.000 dolares al año)
- ¿Cual es la densidad poblacional del condado? (rural o mas de 61.5 personas por milla cuadrada)
- En el 2004, ¿bush arrazo con Kerry?

## 1.4 ¿Cuál es la predicción de clase para el siguiente condado con los siguientes datos: 15% de afroamericanos, 90% de tasa de graduación en la escuela secundaria, ubicado en el Sur, alta pobreza, alta densidad de población?


La predicción es que gane Barack Obama 185 a 36

## 1.5 ¿Cuál es el chance predicha para ese mismo condado anteriormente descrito?

Es 5,1 veces mas probable que voten mayoritariamente en ese condados por Obama que por Hillary

## SVM


Para esta tarea consideramos un conjunto de observaciones sobre una serie de variedades de vino tinto y blanco que implican sus propiedades químicas y su clasificación por parte de los catadores. La industria del vino ha experimentado un gran crecimiento en los últimos tiempos debido al aumento del consumo social. El precio del vino depende de un concepto bastante abstracto de apreciación del vino por parte de los catadores, cuya opinión puede tener un alto grado de variabilidad. El precio del vino depende en cierta medida de este factor tan volátil. Otro factor clave en la certificación y evaluación de la calidad del vino son las pruebas fisicoquímicas, que se realizan en laboratorio y tienen en cuenta factores como la acidez, el nivel de pH, la presencia de azúcar y otras propiedades químicas. Para el mercado del vino, sería interesante que la calidad humana de la cata pudiera relacionarse con las propiedades químicas del vino para que el proceso de certificación y evaluación de la calidad estuviera más controlado.

Se dispone de dos conjuntos de datos, uno de los cuales se refiere al vino tinto y cuenta con 1.599 variedades diferentes, y el otro, al vino blanco, con 4.898 variedades. Todos los vinos se producen en una zona concreta de Portugal. Se recogen datos sobre 12 propiedades diferentes de los vinos, una de las cuales es la calidad, basada en datos sensoriales, y el resto son propiedades químicas de los vinos, como la densidad, la acidez, el contenido de alcohol, etc. Todas las propiedades químicas de los vinos son variables continuas. La calidad es una variable ordinal con una clasificación posible de 1 (peor) a 10 (mejor). Cada variedad de vino es catada por tres catadores independientes y la clasificación final asignada es la mediana de la clasificación dada por los catadores.

Se espera que un modelo predictivo desarrollado a partir de estos datos sirva de orientación a los viñedos en cuanto a la calidad y el precio que se espera obtener de sus productos sin depender en gran medida de la volatilidad de los catadores.


In [64]:
import pandas as pd
import numpy as np

In [65]:
data_r = pd.read_csv('https://raw.githubusercontent.com/prof-tatiana-gutierrez/DatosMD/master/Wine_data_red.csv')
data_w = pd.read_csv('https://raw.githubusercontent.com/prof-tatiana-gutierrez/DatosMD/master/Wine_data_white.csv')

In [66]:
data = data_w.assign(type = 'white')

data = data.append(data_r.assign(type = 'red'), ignore_index=True)
data.sample(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
416,7.4,0.21,0.27,1.20,0.041,27.0,99.0,0.99270,3.19,0.33,9.8,6,white
2281,6.3,0.16,0.22,1.30,0.046,18.0,66.0,0.99307,3.61,0.55,10.3,6,white
2968,8.4,0.31,0.31,0.95,0.021,52.0,148.0,0.99038,2.93,0.32,11.5,5,white
776,8.3,0.14,0.45,1.50,0.039,18.0,98.0,0.99215,3.02,0.56,11.0,6,white
2792,6.8,0.26,0.44,8.20,0.046,52.0,183.0,0.99584,3.20,0.51,9.4,5,white


In [67]:
data.quality.value_counts()

6    2836
5    2138
7    1079
4     216
8     193
3      30
9       5
Name: quality, dtype: int64

In [68]:
data.type.value_counts()

white    4898
red      1599
Name: type, dtype: int64

# Ejercicio 2.1

Mostrar la tabla de frecuencias de la calidad por tipo de vino.

In [69]:
pd.crosstab(data['quality'],data['type'])

type,red,white
quality,,
3,10,20
4,53,163
5,681,1457
6,638,2198
7,199,880
8,18,175
9,0,5


# Ejercicio 2.2

Teniendo en cuenta todas las variables:

* Crear un objetivo binario para cada tipo de vino, haciendo la transformación necesaria para ingresar la información al modelo.
* Crear una SVM que prediga el tipo de vino
* Evalúe con F1, AUC-ROC y log-loss


In [70]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import itertools

data['type'] = data['type'].map({'white':0, 'red':1}).astype(int)
x = data.drop(['type'], axis = 1)
y = data['type']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=69420)

In [71]:
AUC = []
F1 = []
log = []
modSVC = SVC()
modSVC.fit(X_train,y_train)
scv_pred = modSVC.predict(X_test)
AUC.append(metrics.roc_auc_score(y_test, scv_pred))
F1.append(np.sqrt(metrics.f1_score(y_test, scv_pred)))
log.append(np.sqrt(metrics.log_loss(y_test, scv_pred)))

tabla1 = pd.DataFrame([AUC,F1,log],index=['AUC','F1','log']).transpose()
tabla1

,AUC,F1,log
0,0.88879,0.926234,1.501001


# Ejercicio 2.3

* Probar la SVM utilizando los diferentes kernels ('poly', 'rbf', 'sigmoid')
* Evalúe con F1, AUC-ROC y log-loss


# ACLARACIÓN SOBRE LOG-LOSS

Al usar el log loss dentro del kfolds, genera NA, no es seguro el motivo pero probablemente este relacionado a que el SVM con la función SVC de la libreria sklearn no permite el calculo de las probabilidades predictivas, las cuales son usadas por el cross_val_score para generarlos. Por ende, solo se utiliza esta metrica a la hora de sacar la parte de testeo.

In [72]:
kernels = ['poly','rbf','sigmoid']
comb = list(itertools.product(kernels,[2,5,10]))
AUC = []
F1 = []
log = []
for f in kernels:
  modSVC = SVC(kernel=f)
  for i in [2,5,10]:
    cv = KFold(n_splits=i, random_state=69, shuffle=True)
    score = cross_val_score(modSVC, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)
    AUC.append(np.sqrt(np.mean(score)))
    score = cross_val_score(modSVC, X_train, y_train, scoring='f1', cv=cv, n_jobs=-1)
    F1.append(np.sqrt(np.mean(score)))
    score = cross_val_score(modSVC, X_train, y_train, scoring='neg_log_loss', cv=cv, n_jobs=-1)
    log.append(np.sqrt(abs(np.mean(score))))

pd.DataFrame([AUC,F1],index=['AUC','F1'],columns=comb).transpose()

,AUC,F1
"(poly, 2)",0.977330,0.911686
"(poly, 5)",0.978012,0.914223
"(poly, 10)",0.978128,0.914601
"(rbf, 2)",0.963022,0.923227
"(rbf, 5)",0.968331,0.925708
"(rbf, 10)",0.969772,0.925094
"(sigmoid, 2)",0.832980,0.698519
"(sigmoid, 5)",0.832253,0.696802
"(sigmoid, 10)",0.832291,0.696224


In [73]:
kernels = ['poly','rbf']
AUC = []
F1 = []
log = []
for f in kernels:
    modSVC = SVC(kernel=f)
    modSVC.fit(X_train,y_train)
    scv_pred = modSVC.predict(X_test)
    AUC.append(metrics.roc_auc_score(y_test, scv_pred))
    F1.append(np.sqrt(metrics.f1_score(y_test, scv_pred)))
    log.append(np.sqrt(metrics.log_loss(y_test, scv_pred)))

tabla2 = pd.DataFrame([AUC,F1,log],index=['AUC','F1','log'],columns=kernels).transpose()
tabla2

,AUC,F1,log
poly,0.886681,0.907485,1.749487
rbf,0.888790,0.926234,1.501001


# Ejercicio 2.4
* Utilizando el mejor SVM del punto anterior encontrar los parámetros que da el mejor rendimiento teniendo en cuenta los siguientes hiperparámetros
'C': [0.1, 1, 10, 100, 1000], 'gamma': [0.01, 0.001, 0.0001]
* Evalúe cada modelo con F1, AUC-ROC y log-loss

Nota: Para los valores de C y gamma, deben realizar todas las posibles combinaciones entre estas dos variables.

In [74]:
Ci = [0.1, 1, 10, 100, 1000]
Ga = [0.01, 0.001, 0.0001]
comb = list(itertools.product(Ci,Ga))
AUC = []
F1 = []
log = []
for f in Ci:
  for k in Ga:
      modSVC = SVC(kernel= 'rbf', C = f, gamma = k)
      cv = KFold(n_splits=10, random_state=69, shuffle=True)
      score = cross_val_score(modSVC, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)
      AUC.append(np.sqrt(np.mean(score)))
      score = cross_val_score(modSVC, X_train, y_train, scoring='f1', cv=cv, n_jobs=-1)
      F1.append(np.sqrt(np.mean(score)))
      score = cross_val_score(modSVC, X_train, y_train, scoring='neg_log_loss', cv=cv, n_jobs=-1)
      log.append(np.sqrt(abs(np.mean(score))))

pd.DataFrame([AUC,F1],index=['AUC','F1'],columns=comb).transpose()

,AUC,F1
"(0.1, 0.01)",0.983351,0.924577
"(0.1, 0.001)",0.969734,0.924349
"(0.1, 0.0001)",0.960042,0.918487
"(1, 0.01)",0.986321,0.938435
"(1, 0.001)",0.982706,0.931968
"(1, 0.0001)",0.977263,0.926474
"(10, 0.01)",0.991451,0.963718
"(10, 0.001)",0.988985,0.948222
"(10, 0.0001)",0.986419,0.941041
"(100, 0.01)",0.995079,0.978015


In [75]:
Ci = [1000]
Ga = [0.001, 0.0001]
comb = list(itertools.product(Ci,Ga))
AUC = []
F1 = []
log = []
for f in Ci:
  for k in Ga:
      modSVC = SVC(kernel= 'rbf', C = f, gamma = k)
      modSVC.fit(X_train,y_train)
      scv_pred = modSVC.predict(X_test)
      AUC.append(metrics.roc_auc_score(y_test, scv_pred))
      F1.append(np.sqrt(metrics.f1_score(y_test, scv_pred)))
      log.append(np.sqrt(metrics.log_loss(y_test, scv_pred)))

tabla3 = pd.DataFrame([AUC,F1,log],index=['AUC','F1','log'],columns=comb).transpose()
tabla3

,AUC,F1,log
"(1000, 0.001)",0.980501,0.987948,0.635484
"(1000, 0.0001)",0.978029,0.985428,0.699186


# Ejercicio 2.5

Compare los resultados con todos los anteriores modelos y eliga ¿Cuál algoritmo con cuáles hiperparámetros es el que tiene mejor rendimiento según la puntuación F1, AUC-ROC y log-loss?

In [76]:
tabla1

,AUC,F1,log
0,0.88879,0.926234,1.501001


In [77]:
tabla2

,AUC,F1,log
poly,0.886681,0.907485,1.749487
rbf,0.888790,0.926234,1.501001


In [78]:
tabla3

,AUC,F1,log
"(1000, 0.001)",0.980501,0.987948,0.635484
"(1000, 0.0001)",0.978029,0.985428,0.699186


Segun el segundo algoritmo, el mejor Kernel para pronosticar corresponde al rbf, y haciendo uso del mismo, los mejores parametros para C y gamma son 1000 y 0.001 respectivamente.